# Prova 2

Parabéns! Após um bem-sucedido mestrado em métodos matemáticos, vocês foram escolhidos como quants
do fundo Short Term INtelliGence (STING).
O fundo está no Brasil, funding e benchmark são o CDI; as vols são offshore mas pode usar como se fosse
onshore.

De posse de uma série histórica de preços, responda as seguintes questões:

#### Exercício 1:
Como estimar o risco dos portfolios do fundo a partir das séries de preços? Aponte as diferenças entre como
estimar o risco entre os diferentes tipos de séries. Assuma que esse cálculo está sendo feito em 31-Mar-2016.

Primeiro dividi cada conjunto de informações do arquivo em sheets diferentes e carreguei cada um em dataframe:

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as st

path = 'C:/Users/RUBENS_PC/Google Drive/Mestrado/semestre04/Pricing and Hedging/Pricing - Prova 02/BBergValues_split.xlsx';
xlsx = pd.ExcelFile(path)
usdbrl_df = xlsx.parse('USDBRL')
ptax_df = xlsx.parse('PTAX')
ibovespa_df = xlsx.parse('Ibovespa')
di1Jan21_df = xlsx.parse('DI1Jan21')
cdi_df = xlsx.parse('CDI')
cds_df = xlsx.parse('CDS')
volATM1M_df = xlsx.parse('VolATM1M')
volATM3M_df = xlsx.parse('VolATM3M')
volATM12M_df = xlsx.parse('VolATM12M')
swapPreCDI1M_df = xlsx.parse('SwapPreCDI1M')
swapPreCDI3M_df = xlsx.parse('SwapPreCDI3M')
swapPreCDI12M_df = xlsx.parse('SwapPreCDI12M')
fwd1M_df = xlsx.parse('FWD1M')
fwd3M_df = xlsx.parse('FWD3M')
fwd12M_df = xlsx.parse('FWD12M')


In [18]:
usdbrl_df['Date'][586]

Timestamp('2016-03-31 00:00:00')

### Exercício 3:
Em 30-Jun-2015 vários traders compraram opções de compra de USD (venda de BRL) ATMF para diferentes
prazos. Use os dados anexos, interpolação linear para spot e forwards, e a interpolação adequada para vols
ATMF (vol forward constante, variância efetiva é a soma das variâncias efetivas) para estimar os resultados
finais e perdas máximas de cada um dos traders.

Preparando os métodos baseados no material de finanças do segundo semestre:

In [19]:
# Black-Scholes formula for fx options
# phi: 1 = call e -1 = put;
# Q: quantity of CCY2 contracts;
# M: multiplier
# fxfwd: is the CCY1/CCY2 fx forward price seen at t for settlement date Tsettle
# K: strike
# vol: effective variance from t to Tex
# fwdDf: forward discount factor in CCY1 seen at date t from fx option premium payment date tpay to fx option payoff date Tpay
# t: computed from t to fx option expiry date Tex

def fxBsv(phi, Q, M, fxfwd, K, vol, fwdDf, t):
    efv=t*(vol**2)
    # calculate d1 and d2
    d1=(np.log(fxfwd/K)+(efv/2))/(np.sqrt(efv))
    d2=(np.log(fxfwd/K)-(efv/2))/(np.sqrt(efv))
    # calculate N(d1) and N(d2)
    Nd1=st.norm.cdf(phi*d1)
    Nd2=st.norm.cdf(phi*d2)
    # calculate premium
    pr=phi*Q*M*(fxfwd*Nd1-K*Nd2)*fwdDf
    # calculate delta
    de=phi*Nd1
        
    return [pr, de]

def fxPayoff(phi, Q, M, K, ptax):
    de = 0
    pr = Q*M*max(phi*(ptax-K), 0)
    return [pr, de]

call=1
put=-1

(a) O primeiro comprou uma opção de 1m e carregou sem hedge até o final.

Vamos calcular o preço e o delta da call de 1M no dia 30-Jun-2015 (data referente ao índice 389 dos dataframes contruídos):

In [22]:
Q = 1
M = 1
fxfwd = fwd1M_df['PX_LAST'][389] # Forward 1 mês (On)
K = fwd1M_df['PX_LAST'][389] # ATMF
vol = volATM1M_df['PX_LAST'][389] # Vol Implícita ATM 1 mês (Off)
fwdDf = swapPreCDI1M_df['PX_LAST'][389] # Swap Pré-CDI 1 mês
t = 1/12

fwd1M_df['Date'][389], fxfwd, K, vol, fwdDf

(Timestamp('2015-06-30 00:00:00'),
 3.1364000000000001,
 3.1364000000000001,
 16.111999999999998,
 13.674300000000001)

In [21]:
test1 = fxBsv(call, Q, M, fxfwd, K, vol, fwdDf, t)
test1

[42.028525972052883, 0.98997916603196667]